In [294]:
import pandas as pd
import numpy as np

In [295]:
file_srna_dat = '../../data/res/04_02_19/counts.fixed.dat.0.05.csv'
file_ann = '../../data/genome/annotations/genes.gff3'
file_deg = '../../data/res/04_02_19/DEG-normalized.csv'
file_output = '../../data/res/DEGX_sRNA_DAT.csv'

In [296]:
#load sRNA loci DATs

In [297]:
df_srna_dat = pd.read_csv(file_srna_dat, index_col=False, sep=',', comment='#')
print(len(df_srna_dat.index))

280


In [298]:
df_srna_dat.sample(3)

,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,X21dpiR1T.trimmed,X21dpiR3T.trimmed
63,Cluster_33427 _ Un:443605205-443605719,166.613480,1.167749,0.310674,3.758762,0.000171,0.007948,110,95.174791,239.457941,221.821187
60,Cluster_28286 _ 7A:379928277-379929939,590.855043,0.955348,0.253146,3.773901,0.000161,0.007722,419,385.341836,886.620415,672.457921
70,Cluster_5516 _ 2A:652328873-652329011,285.697430,-1.245848,0.339714,-3.667346,0.000245,0.010140,539,264.632345,175.289473,163.867904


In [299]:
df_srna_dat['loci'] = df_srna_dat['Gene'].str.split(' _ ').str[0]
df_srna_dat['seqname'] = df_srna_dat['Gene'].str.split(' _ ').str[1]
df_srna_dat['start'] = df_srna_dat['seqname'].str.split('-').str[0]
df_srna_dat['end'] = df_srna_dat['seqname'].str.split('-').str[1]
df_srna_dat['seqname'] = df_srna_dat['start'].str.split(':').str[0]
df_srna_dat['start'] = df_srna_dat['start'].str.split(':').str[1]

In [300]:
df_srna_dat.sample(3)

,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,X21dpiR1T.trimmed,X21dpiR3T.trimmed,loci,seqname,start,end
206,Cluster_33306 _ Un:416933289-416933469,571.549023,0.863779,0.284257,3.038729,0.002376,0.034773,481,329.629764,867.056857,608.509471,Cluster_33306,Un,416933289,416933469
239,Cluster_33484 _ Un:455961952-455962668,353.314137,0.838498,0.287784,2.913635,0.003572,0.044587,309,197.313591,452.309444,454.633513,Cluster_33484,Un,455961952,455962668
148,Cluster_27727 _ 7A:72700594-72700677,392.916260,-0.880354,0.272316,-3.232847,0.001226,0.024311,592,425.965223,262.934209,290.765609,Cluster_27727,7A,72700594,72700677


In [301]:
df_srna_dat = df_srna_dat[['loci','seqname','start','end','pvalue','padj','log2FoldChange']]
df_srna_dat.head(4)

,loci,seqname,start,end,pvalue,padj,log2FoldChange
0,Cluster_17433,4D,236302201,236302457,4.497539e-17,1.339817e-13,1.991005
1,Cluster_16619,4B,600374555,600374577,1.289745e-15,1.921076e-12,2.254593
2,Cluster_9813,3A,149826050,149826090,6.079758e-12,3.260401e-09,10.206085
3,Cluster_15102,4A,611586002,611586328,6.566769e-12,3.260401e-09,1.826796


In [302]:
#genes DEG

In [303]:
df_deg = pd.read_csv(file_deg, index_col=False, sep=',', comment='#')
print(len(df_deg.index))
df_deg['gene'] = df_deg['gene'].str.split('gene:').str[1]
df_deg.drop("Unnamed: 0", axis=1, inplace=True)

3019


In [304]:
df_ann = pd.read_csv(file_ann, index_col=False, sep='\t', comment='#', header=None)
df_ann.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']
df_ann['gene'] = df_ann.attribute.str.split('gene:').str[1]
df_ann['gene'] = df_ann.gene.str.split(';').str[0]
df_ann = df_ann[['gene','seqname','start','end', 'strand']]
print(len(df_ann.index))


120744


In [305]:
df_deg_pos = pd.merge(df_deg, df_ann, on='gene')
df_deg_pos = df_deg_pos[['gene','log2FoldChange','pvalue','padj','seqname','start','end','strand']]

In [306]:
df_deg_pos.head(3)

,gene,log2FoldChange,pvalue,padj,seqname,start,end,strand
0,TraesCS2B02G286300,-2.445058,1.703566e-27,5.369742e-23,2B,394317533,394318758,+
1,TraesCS2B02G336800,2.909847,1.882766e-27,5.369742e-23,2B,481048634,481054363,+
2,TraesCS6D02G241200,-2.320181,2.645301e-25,5.029687e-21,6D,343557954,343558810,-


In [307]:
#GENEs

In [308]:
#merge with genes

In [309]:
df_deg_pos['cluster_name'] = ''
df_deg_pos['cluster_seqname'] = ''
df_deg_pos['cluster_start'] = ''
df_deg_pos['cluster_end'] = ''
df_deg_pos['cluster_position'] = ''
df_deg_pos['cluster_padj'] = ''
df_deg_pos['cluster_pvalue'] = ''
df_deg_pos['cluster_log2FoldChange'] = ''

In [310]:
howfar = 3000
rows = []
for k_gene, v_gene in df_deg_pos.iterrows():
    for k_srna,v_srna in df_srna_dat[df_srna_dat.seqname == v_gene.seqname].iterrows():
        #overlaps with gene
        if int(v_srna.end) >= int(v_gene.start) and int(v_srna.start) <= int(v_gene.end):
            df_deg_pos.at[k_gene, 'cluster_name'] = v_srna.loci
            df_deg_pos.at[k_gene, 'cluster_seqname'] = v_srna.seqname
            df_deg_pos.at[k_gene, 'cluster_start'] = v_srna.start
            df_deg_pos.at[k_gene, 'cluster_end'] = v_srna.end
            df_deg_pos['cluster_padj'] = v_srna.padj
            df_deg_pos['cluster_pvalue'] = v_srna.pvalue
            df_deg_pos['cluster_log2FoldChange'] = v_srna.log2FoldChange
            df_deg_pos.at[k_gene, 'cluster_position'] = 'overlaps'
            rows.append(df_deg_pos.iloc[k_gene])
        else:
            #upstream (promoter)
            if v_gene.strand == '+':
                start = max(int(v_gene.start) - howfar, 0)
                end = int(v_gene.start)
            else:
                start = int(v_gene.end)
                end = int(v_gene.end) + howfar
            if int(v_srna.end) >= start and int(v_srna.start) <= end:
                df_deg_pos.at[k_gene, 'cluster_name'] = v_srna.loci
                df_deg_pos.at[k_gene, 'cluster_seqname'] = v_srna.seqname
                df_deg_pos.at[k_gene, 'cluster_start'] = v_srna.start
                df_deg_pos.at[k_gene, 'cluster_end'] = v_srna.end
                df_deg_pos.at[k_gene, 'cluster_position'] = 'promoter'
                df_deg_pos['cluster_padj'] = v_srna.padj
                df_deg_pos['cluster_pvalue'] = v_srna.pvalue
                df_deg_pos['cluster_log2FoldChange'] = v_srna.log2FoldChange
                rows.append(df_deg_pos.iloc[k_gene])
            else:
                #downstream
                if v_gene.strand == '+':
                    end = int(v_gene.end) + howfar
                    start = int(v_gene.end)
                else:
                    end = int(v_gene.start)
                    start = max(int(v_gene.start) - howfar,0)
                if int(v_srna.end) >= start and int(v_srna.start) <= end:
                    df_deg_pos.at[k_gene, 'cluster_name'] = v_srna.loci
                    df_deg_pos.at[k_gene, 'cluster_seqname'] = v_srna.seqname
                    df_deg_pos.at[k_gene, 'cluster_start'] = v_srna.start
                    df_deg_pos.at[k_gene, 'cluster_end'] = v_srna.end
                    df_deg_pos.at[k_gene, 'cluster_position'] = 'downstream'
                    df_deg_pos['cluster_padj'] = v_srna.padj
                    df_deg_pos['cluster_pvalue'] = v_srna.pvalue
                    df_deg_pos['cluster_log2FoldChange'] = v_srna.log2FoldChange
                    rows.append(df_deg_pos.iloc[k_gene])
                

In [311]:
df_res = pd.DataFrame(rows)

In [312]:
df_res

,gene,log2FoldChange,pvalue,padj,seqname,start,end,strand,cluster_name,cluster_seqname,cluster_start,cluster_end,cluster_position,cluster_padj,cluster_pvalue,cluster_log2FoldChange
2992,TraesCS4A02G323700,0.54068,0.002577,0.049104,4A,611579524,611583216,+,Cluster_15102,4A,611586002,611586328,downstream,3.260401e-09,6.566769e-12,1.826796


In [313]:
len(df_res[df_res.cluster_name != ''].index)

1

In [314]:
df_res.to_csv(file_output, index=None, )
file_output

'../../data/res/DEG_X_sRNA_DAT.21.csv'